In [1]:
# Load main packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import getpass # For identifying user/directory structure
import networkx as nx
from sklearn import linear_model
%matplotlib inline

# Load local files
import sys # For reading files in other directories
if(getpass.getuser() == 'rockc_000'):
    sys.path.insert(0, 'C:/Users/rockc_000/Documents/GitHub/GithubSandbox/Python/NCAA Scripts')
    dir = 'C:/Users/rockc_000/Documents/Personal Files/Kaggle/NCAA'
if(getpass.getuser() == 'josh'):
    sys.path.insert(0, '/home/josh/Documents/Github/GithubSandbox/Python/NCAA Scripts')
    dir = '/home/josh/Documents/Personal/Kaggle/NCAA'
from make_submission import make_submission

In [2]:
# Read in data
seasons = pd.read_csv(dir + '/Seasons.csv')
teams = pd.read_csv(dir + '/Teams.csv')
regular_compact_results = pd.read_csv(dir + '/RegularSeasonCompactResults.csv')
regular_detailed_results = pd.read_csv(dir + '/RegularSeasonDetailedResults.csv')
tourney_compact_results = pd.read_csv(dir + '/TourneyCompactResults.csv')
tourney_detailed_results = pd.read_csv(dir + '/TourneyDetailedResults.csv')
tourney_seed = pd.read_csv(dir + '/TourneySeeds.csv')
tourney_slots = pd.read_csv(dir + '/TourneySlots.csv')
sample_submission = pd.read_csv(dir + '/SampleSubmission.csv')

In [3]:
# Look at the first few rows of each dataset, and their dimensions
# sample_submission.iloc[np.arange(5)]
nrow = 4
print('seasons:\n', seasons.head(nrow), '\nDimension: ', seasons.shape)
print('teams:\n', teams.head(nrow), '\nDimension: ', teams.shape)
print('regular_compact_results:\n', regular_compact_results.head(nrow),
      '\nDimension: ', regular_compact_results.shape)
print('regular_detailed_results:\n', regular_detailed_results.head(nrow),
      '\nDimension: ', regular_detailed_results.shape)
print('tourney_compact_results:\n', tourney_compact_results.head(nrow),
      '\nDimension: ', tourney_compact_results.shape)
print('tourney_detailed_results:\n', tourney_detailed_results.head(nrow),
      '\nDimension: ', tourney_detailed_results.shape)
print('tourney_seed:\n', tourney_seed.head(nrow),
      '\nDimension: ', tourney_seed.shape)
print('tourney_slots:\n', tourney_slots.head(nrow),
      '\nDimension: ', tourney_slots.shape)
print('sample_submission:\n', sample_submission.head(nrow),
      '\nDimension: ', sample_submission.shape)

seasons:
    Season     Dayzero Regionw    Regionx    Regiony    Regionz
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988  11/02/1987    East    Midwest  Southeast       West 
Dimension:  (32, 6)
teams:
    Team_Id    Team_Name
0     1101  Abilene Chr
1     1102    Air Force
2     1103        Akron
3     1104      Alabama 
Dimension:  (364, 2)
regular_compact_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985      20   1228      81   1328      64    N      0
1    1985      25   1106      77   1354      70    H      0
2    1985      25   1112      63   1223      56    H      0
3    1985      25   1165      70   1432      54    H      0 
Dimension:  (139920, 8)
regular_detailed_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003      10   1104      68   1328      6

## Pagerank algorithm on historical wins

In [40]:
# Initialize DataFrame with one (meaningless) row
pagerank_score = pd.DataFrame({'Season': [1900], 'Team': [0], 'Score': [0]})
for season in seasons['Season']:
    G = nx.DiGraph()
    edges = regular_compact_results.ix[regular_compact_results['Season'] == season,
                                       ('Wteam', 'Lteam')]
    for row in np.arange(edges.shape[0]):
        G.add_edge(edges.iloc[row][1], edges.iloc[row][0])
    # Calculate the page-rank using the networkx package
    out = nx.pagerank(G)
    # Convert the page-rank scores into a DataFrame.
    out = pd.DataFrame.from_dict(Graphs[0], orient='index')
    out.rename(columns={0: 'Score'}, inplace=True)
    out['Team'] = out.index
    out['Season'] = season
    pagerank_score = pagerank_score.append(out)
pagerank_score = pagerank_score.ix[pagerank_score['Team'] != 0]

## Aggregated Statistics

In [104]:
win_count = regular_compact_results.groupby(('Season', 'Wteam'))
win_count = win_count['Daynum'].count()
win_pts = regular_compact_results.groupby(('Season', 'Wteam'))
win_pts = win_pts['Wscore'].sum()
lose_pts = regular_compact_results.groupby(('Season', 'Lteam'))
lose_pts = lose_pts['Lscore'].sum()
win_pts.index.set_names(('Season', 'Team'), inplace=True)
lose_pts.index.set_names(('Season', 'Team'), inplace=True)
total_pts = pd.merge(pd.DataFrame(win_pts),
                     pd.DataFrame(lose_pts),
                     left_index=True, right_index=True)
total_pts['total_score'] = total_pts['Wscore'] + total_pts['Lscore']
total_pts

Wscore  Lscore  total_score
Season Team                             
1985   1102     355    1159         1514
       1103     632     772         1404
       1104    1514     541         2055
       1106     751     968         1719
       1108    1631     444         2075
       1109      54    1238         1292
       1110     550    1186         1736
       1111     708     903         1611
       1112    1265     531         1796
       1113     824    1004         1828
       1114    1306     851         2157
       1116    1448     708         2156
       1117     727     874         1601
       1119    1039     817         1856
       1120    1308     732         2040
       1121     548    1115         1663
       1122     387    1081         1468
       1123    1093    1132         2225
       1124     637    1246         1883
       1126     419    1259         1678
       1129     845     833         1678
       1130    1300     647         1947
       1131     997     919         1916
       1132     791     961         1752
       1133    1163     806         1969
       1134     842     809         1651
       1135     576    1160         1736
       1137    1099     676         1775
       1139    1117     519         1636
       1140    1119    1056         2175
...             ...     ...          ...
2015   1433    1992     545         2537
       1434    1780     272         2052
       1435    1335     812         2147
       1436    1179     786         1965
       1437    2474     119         2593
       1438    1904     187         2091
       1439     824    1332         2156
       1440     789    1369         2158
       1441     940    1153         2093
       1442     432    1150         1582
       1443    1324     768         2092
       1444    1369     819         2188
       1447     685    1284         1969
       1448     978    1231         2209
       1449    1150     978         2128
       1450    1094    1095         2189
       1451     747    1080         1827
       1452    1786     580         2366
       1453    1560     470         2030
       1454     851     973         1824
       1455    1888     238         2126
       1456    1368     793         2161
       1457    1199     879         2078
       1458    2259     185         2444
       1459    1776     332         2108
       1460     635    1197         1832
       1461    1519     447         1966
       1462    1586     915         2501
       1463    1353     612         1965
       1464     629    1425         2054

[9818 rows x 3 columns]